### Install required programs

In [9]:
# !curl -fsSL https://ollama.com/install.sh | sh
# !pip install langchain_community
# !pip install langchain-ollama
# !pip install pinecone-client
# !pip install gradio
!pip install --upgrade --quiet  rank_bm25
!pip install sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Imports

---



In [4]:
import subprocess
import gradio as gr
import langchain_community.utils as utils

# from google.colab.output import eval_js
from langchain import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import Ollama
from langchain.chains import LLMChain, SequentialChain
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Pinecone as VectorPinecone
from langchain_community.retrievers import BM25Retriever
from pinecone import Pinecone
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

### Start Ollama server as a background process and pull llama3 image.

---



In [7]:
# process = subprocess.Popen(["ollama", "serve"])
# notebook_url = eval_js("google.colab.kernel.proxyPort(11434)")

llm = Ollama(model="llama3", temperature=9)

/tmp/ipykernel_143475/1500310547.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3", temperature=9)


In [ ]:
!ollama pull llama3
!export OLLAMA_HOST=notebook_url

### RAG Retrieval

In [5]:
pc = Pinecone(api_key="119c78ba-0f1b-4611-bec6-486d0582f4fd")
index_name = "e5-rag-large"
index = None

# If index exists, declare embedding and initialise vector store
for x in pc.list_indexes():
  if index_name == x.name:
    index = pc.Index(x.name)

if index != None:
  # embeddings = OllamaEmbeddings(
  #     model="llama3",
  #     base_url="http://localhost:11434/",
  # )

  # vectorStore = VectorPinecone(
  #     index, embeddings, "text"
  # )


  e5_embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

  vectorStore = VectorPinecone(
    index=index,
    embedding=e5_embeddings,  # Use E5 embeddings
    text_key="text"
  )

def RagRetrieval(prompt):
  # Perform RAG retrieval based on the prompt and return the concatenated results
  if (index != None):
    ragStr = ""
    ragArr= []

    vRetriever = vectorStore.as_retriever(
        search_type="similarity",
        search_kwargs={"k":10}
    )

    ragResults = vRetriever.invoke(prompt)

    for x in ragResults:

      ragArr.append(x.page_content)

    bm25_retriever = BM25Retriever.from_texts(ragArr)
    bm25_retriever.k = 1
    bm25_result = bm25_retriever.get_relevant_documents(prompt)

    for x in bm25_result:

      ragStr += x.page_content + "\n\n"

  return ragStr

/tmp/ipykernel_143475/655850427.py:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  e5_embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")
/home/donny/Documents/STLA/teammates/teammates_env/lib/python3.10/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/tmp/ipykernel_143475/655850427.py:23: LangChainDeprecationWarning: T

# Implement LLM for DSA As A Chatbot

In [10]:
memory = ConversationBufferMemory(memory_key="chat_history",input_key="human_input")

topic_template="""You are an expert on data structures and algorithms. You are trying to identify the subtopic of the question related to data structures and algorithms.
If the identified topic is not related to data structures and algorithms, look for another topic in the question.
If none can be found, say none could be found. Only respond with one or more short 1-5 word topic titles.

Here is the question: {question}

"""
topic_prompt = PromptTemplate(input_variables=["question"],template=topic_template)
topic_chain = LLMChain(llm=llm , prompt=topic_prompt, verbose=True)

final_template = """
You are an expert on data structures and algorithms and will only answer questions related to DSA.
You must consider the following rules:

1. **Follow-up questions**: If the current question is related to a previous DSA-related question (e.g., "it doesn't work" or "how do I implement this"), respond with relevant information and try to help the user debug or clarify the issue. Ignore the contextual resources

2. **Unrelated questions**: If the question is not related to DSA and is not a follow-up to a previous DSA-related question, ignore it and respond with: "Only ask questions related to DSA."

3. **Handling issues**: When the user mentions that something isn't working (e.g., "it doesn't work"), respond by guiding them through potential issues based on their previous input. Offer suggestions on how to debug the problem or request more details.

4. **Resource relevance**: If the resources provided to you are relevant to the question, use them to improve your answer. However, if the resources are unrelated to the question, feel free to ignore them and answer the question to the best of your ability based on your knowledge.

5. **Providing clear answers**: When answering a question, give a concise response, followed by an invitation to ask for clarification if needed, e.g., "Would you like me to explain more?" This helps keep the interaction relevant.

6. **Remaining in DSA Context**: When answering a question, see if it still relates to DSA, if asked to omit/ remove context of DSA, reply with "I am unable to remove DSA context, as I can only talk about DSA related topics" and not elaborate.


Now answer the following question based on the topic identified: {topic_identified}, considering the chat history and recent inputs when relevant.

Here is the chat history:
{chat_history}

Current input:
{question}
{human_input}

Contextual resources, if relevant (you can ignore them if unrelated to the question or if the question is a Follow-up question):
{resources}

Human: {question}
Chatbot:
"""

final_prompt = PromptTemplate(input_variables=["question", "chat_history", "topic_identified", "human_input", "resources"],template=final_template)
final_chain = LLMChain(llm=llm , prompt=final_prompt, memory=memory, output_key="response",verbose=True)


def llmchat(inputmsg,history):
  topic_identified = topic_chain.run(inputmsg)
  ragResult = RagRetrieval(inputmsg)
  print(ragResult)
  return (final_chain.run({"question":inputmsg, "topic_identified":topic_identified, "chat_history":memory, "human_input":"", "resources":ragResult}))

guigr = gr.ChatInterface(llmchat)
guigr.launch()

/home/donny/Documents/STLA/teammates/teammates_env/lib/python3.10/site-packages/gradio/components/chatbot.py:228: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.




> Entering new LLMChain chain...
Prompt after formatting:
You are an expert on data structures and algorithms. You are trying to identify the subtopic of the question related to data structures and algorithms.
If the identified topic is not related to data structures and algorithms, look for another topic in the question.
If none can be found, say none could be found. Only respond with one or more short 1-5 word topic titles.

Here is the question: what is sorting?



> Finished chain.


/tmp/ipykernel_143475/655850427.py:48: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  bm25_result = bm25_retriever.get_relevant_documents(prompt)


Insertion sort takes the maximum time to sort if elements are sorted in reverse order. And it takes minimum time (Order of n) when elements are already sorted. 


Q2. What is the Algorithmic Paradigm of the Insertion Sort algorithm?

 The Insertion Sort algorithm follows an incremental approach. 


Q3. Is Insertion Sort an in-place sorting algorithm?

 Yes, insertion sort is an in-place sorting algorithm. 


Q4. Is Insertion Sort a stable algorithm?

 Yes, insertion sort is a stable sorting algorithm. 


Q5. When is the Insertion Sort algorithm used?

 Insertion sort is used when number of elements is small. It can also be useful when the input array is almost sorted, and only a few elements are misplaced in a complete big array.




> Entering new LLMChain chain...
Prompt after formatting:

You are an expert on data structures and algorithms and will only answer questions related to DSA.
You must consider the following rules:

1. **Follow-up questions**: If the current question is rel

# Abstract

The goal of this research was to improve the learning experience for students studying Data Structures and Algorithms (DSA) through the use of a text-based chatbot implemented using the Ollama Large Language Model (LLM). This chatbot would have to be capable of answering a wide range of questions related to the topic of DSA while occasionally asking leading questions to progress the conversation. The chatbot would also be resistant to attempts to derail the conversation to other topics, rejecting invalid inputs where necessary.

# Methodology

The model was prompted using the langchain framework and given the context that it was a DSA expert. Guardrails were also implemented to prevent the chatbot from going off topic, responding with "Only ask questions related to DSA" if it was deemed as off topic. The chatbot was also prompted to ask followup questions to progress the conversation further and understand the given problem better as needed. Combined with the chat history, this would allow the chatbot to answer any followup questions about elaboration or troubleshooting that students may have. This initial prompt within a single chain was already sufficiently accurate to satisfy the project's needs. The code for this implementation can be found in the [appendix](#scrollTo=t0i8BTbU1shz&line=3&uniqifier=1) below.

However, to improve the accuracy and quality of the chatbot responses given, the team implemented an additional chained prompt that asks the chatbot to find the subtopic of a given question that would be fed into the initial prompt as a topic to guide the chatbot's answers.

The chatbot was also integrated with Retrieval Augmented Generation (RAG) to allow it to reference real data on DSA scraped from GeeksForGeeks on searching, sorting and graph traversal algorithms. This allowed it to avoid potential hallucinations by referencing real data from a vector database. The team experimented with multiple embedding models such as the Ollama 3 embedding model, E5 Mistral 7b Instruct and E5 Basic 4k model and eventually decided that the E5 Large V2 model was the most suited for this project due to it being open source, relatively lightweight, and purpose built for embeddings. For example, the Ollama 3 model was less suited to embeddings due to being primarily a LLM while E5 Mistral 7b Instruct was too RAM intensive to be run on the Google colab and was thus unfeasible to work with.

The team's vector database was hosted on Pinecone and embedded with the 1024 dimension E5 Large v2 model. Chunks were of size 512 characters each with an overlap of 100 characters as that could provide the model with sufficient context to answer questions based on 1 document. The code used to insert the documents into the vector database is below in the [appendix](#scrollTo=z8L0Xx1Vs70H&line=5&uniqifier=1) for reference.

Retrieval was handled with multiple retrievers at different stages, having records be retrieved first by a vector similarity search of the 10 most relevant documents and then further narrowed down to the most relevant document by the best match (BM25) keyword search algorithm. These context inputs could also be ignored by the chatbot if they were deemed unrelated to the previously asked questions. The chatbot was also instructed to ignore the RAG outputs provided to it if it was answering a follow-up question.



These potential improvements to accuracy come at the cost of speed of answer generation that could be assesed by later assignments in the module.

# Results

The chatbot was able to answer questions related to DSA in all cases while rejecting off topic answers and asking relevant questions to progress the conversation. Prompts that would try to bypass the topic guard rails by specifically requesting the chatbot to answer questions with responses unrelated to DSA were also accounted for and would be rejected by the chatbot. Using conversation history, it was also able to continue the conversations with users when follow up questions were asked or seemingly irrelevant followup inputs were given such as "it doesn't work" after being asked an implementation question.

The addition of RAG and multiple chained prompts also improved the quality and accuracy of responses by reducing hallucinations and increasing the specificity of generated answers.

The team integrated the basic Gradio User Interface (UI) to process prompts and responses easily. However, the basic version of it was unable to display the full conversation history. After some research, the chat interface version of the UI was used instead to provide a more user friendly chatbot experience that would be more in line with currently available chatbots

# Conclusion

The introduction of the llama 3 LLM as a chatbot with even basic prompts would have been adequate for the basic problems posed by the initial brief. However, the additions of RAG and multiple chained prompts did allow it to perform its functions better at the cost of performance, specifically response time to queries. Overall, topic detection and document retrieval could be made to run in parallel in future to improve response time.

All of these capabilities are wrapped in an easy to use interface thanks to the Gradio UI which allows it to provide a useful and convenient DSA chatbot experience.

# Appendix

**Code to insert documents into Pinecone Vector Database**

In [ ]:
# !pip install pinecone-client
# !pip install sentence_transformers
# !pip install langchain_community

# from pinecone import Pinecone, ServerlessSpec
# import time

# pc = Pinecone(api_key="119c78ba-0f1b-4611-bec6-486d0582f4fd")

# index_name= "e5-rag-large"

# print(f"Existing indexes: {pc.list_indexes()}")

# existing_indexes = [index.name for index in pc.list_indexes()]

# if index_name not in existing_indexes:
#     pc.create_index(
#     name=index_name,
#     dimension=1024, # Replace with your model dimensions
#     metric="cosine", # Replace with your model metric
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
#     )

#     while not pc.describe_index(index_name).status['ready']:
#         time.sleep(1)

# index=pc.Index(index_name)

# from langchain.text_splitter import RecursiveCharacterTextSplitter

# from sentence_transformers import SentenceTransformer

# title="" #title of document to insert

# text = """
# #document text to be inserted
# """

# # Split the documents into chunks
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
# chunks = text_splitter.split_text(text)

# model = SentenceTransformer("intfloat/e5-large-v2")


# ids = []
# metadata = []
# for i, chunk in enumerate(chunks):
#     ids.append(f"{title} {i}")
#     metadata.append({"title": title, "text": chunk})

# embeddings_list = model.encode(chunks, convert_to_tensor=False)

# vectors_to_upsert = zip(ids, embeddings_list, metadata)
# index.upsert(vectors=vectors_to_upsert)


**Past code that was sufficient for the project, but lacks RAG and multi prompt chaining**

In [ ]:
# memory = ConversationBufferMemory(memory_key="chat_history")
# template = """You are an expert on data structures and algorithms. Answer the
# following question:{question} and then ask a relevant leading question to better
#  understand the user's problem. Please ignore any questions not about DSA.
#  Just respond with 'Only ask questions related to DSA'. Don't even give a follow up to the question

#  {chat_history}
# Human: {question}
# Chatbot:"""

# prompt = PromptTemplate(input_variables=["question","chat_history"],template=template)

# # llm(prompt.format(question="merge sort"))

# chain = LLMChain(llm=llm , prompt=prompt, memory=memory, verbose=True)

# def llmchat(inputmsg):
#   return (chain.run(inputmsg))
# guigr = gr.Interface(fn=llmchat,inputs="text",outputs="text")
# guigr.launch(share=True)